In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 📥 Data Collection & Processing\n",
    "### ERA5 Climate Data for Nepal River Basins\n",
    "\n",
    "This notebook demonstrates:\n",
    "- Fetching ERA5 precipitation and runoff data\n",
    "- Converting to river discharge\n",
    "- Calibrating rating curves for water level estimation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "from datetime import datetime, timedelta\n",
    "import matplotlib.pyplot as plt\n",
    "from src.config import Config\n",
    "from src.data_loader import ERA5DataLoader\n",
    "\n",
    "plt.style.use('seaborn-v0_8-darkgrid')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Select Station"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Choose a river station\n",
    "station_key = 'koshi_chatara'\n",
    "station_config = Config.TARGET_STATIONS[station_key]\n",
    "\n",
    "print(f\"Station: {station_config['name']}\")\n",
    "print(f\"River: {station_config['river']}\")\n",
    "print(f\"Drainage Area: {station_config['drainage_area']:,} km²\")\n",
    "print(f\"Location: {station_config['lat']:.2f}°N, {station_config['lon']:.2f}°E\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Load ERA5 Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize data loader\n",
    "loader = ERA5DataLoader(station_config, years_back=5)\n",
    "\n",
    "# Fetch data (uses cache if available)\n",
    "data = loader.fetch_era5_data()\n",
    "\n",
    "print(f\"\\nLoaded {len(data):,} hourly records\")\n",
    "print(f\"Date range: {data['timestamp'].min()} to {data['timestamp'].max()}\")\n",
    "data.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Data Overview"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Summary statistics\n",
    "print(\"\\n=== DISCHARGE STATISTICS ===\")\n",
    "print(f\"Mean: {data['discharge_cumecs'].mean():.2f} m³/s\")\n",
    "print(f\"Median: {data['discharge_cumecs'].median():.2f} m³/s\")\n",
    "print(f\"90th percentile: {data['discharge_cumecs'].quantile(0.90):.2f} m³/s\")\n",
    "print(f\"Max: {data['discharge_cumecs'].max():.2f} m³/s\")\n",
    "\n",
    "print(\"\\n=== WATER LEVEL STATISTICS ===\")\n",
    "print(f\"Mean: {data['water_level_m'].mean():.2f} m\")\n",
    "print(f\"90th percentile: {data['water_level_m'].quantile(0.90):.2f} m\")\n",
    "print(f\"Max: {data['water_level_m'].max():.2f} m\")\n",
    "print(f\"Flood stage: {station_config['flood_stage']:.2f} m\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Visualize Time Series"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot last year of data\n",
    "recent = data[data['year'] == data['year'].max()]\n",
    "\n",
    "fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)\n",
    "\n",
    "# Discharge\n",
    "ax1.plot(recent['timestamp'], recent['discharge_cumecs'], linewidth=0.8)\n",
    "ax1.set_ylabel('Discharge (m³/s)', fontsize=11)\n",
    "ax1.set_title(f\"{station_config['name']} - Recent Year\", fontsize=13, weight='bold')\n",
    "ax1.grid(True, alpha=0.3)\n",
    "\n",
    "# Water Level\n",
    "ax2.plot(recent['timestamp'], recent['water_level_m'], linewidth=0.8, color='steelblue')\n",
    "ax2.axhline(station_config['flood_stage'], color='orange', linestyle='--', label='Flood Stage')\n",
    "ax2.axhline(station_config['moderate_flood'], color='red', linestyle='--', label='Moderate Flood')\n",
    "ax2.set_ylabel('Water Level (m)', fontsize=11)\n",
    "ax2.set_xlabel('Date', fontsize=11)\n",
    "ax2.legend()\n",
    "ax2.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Rating Curve Validation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot discharge vs water level relationship\n",
    "plt.figure(figsize=(10, 6))\n",
    "plt.scatter(data['discharge_cumecs'], data['water_level_m'], alpha=0.1, s=5)\n",
    "plt.axhline(station_config['flood_stage'], color='orange', linestyle='--', label='Flood Stage')\n",
    "plt.xlabel('Discharge (m³/s)', fontsize=11)\n",
    "plt.ylabel('Water Level (m)', fontsize=11)\n",
    "plt.title('Rating Curve: Discharge vs Water Level', fontsize=13, weight='bold')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.show()\n",
    "\n",
    "print(f\"\\nRating curve successfully calibrated:\")\n",
    "print(f\"90th percentile discharge → {station_config['flood_stage']:.2f}m (flood stage)\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ Summary\n",
    "\n",
    "- ERA5 data successfully fetched and processed\n",
    "- Rating curve calibrated to station characteristics\n",
    "- Water levels calculated from precipitation and runoff\n",
    "- Data ready for flood analysis and forecasting"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}